In [1]:
import psycopg2
from psycopg2 import sql
!ln -s /var/run/postgresql/.s.PGSQL.5432 /tmp/.s.PGSQL.5432

In [59]:
conn = psycopg2.connect(database = "zambiaLFSmapping", user = "ram22")
cur = conn.cursor()

In [61]:
# Table of population counts for each ward for relevant age group
cur.execute("""
    create table if not exists censuswardcounts1564 as
      select max(dist) as dist, max(const) as const, max(ward) as ward,
        wardid, count(wardid) as population1564
      from censuslabor
      where p2_membership <= 2 
        and p5_age >= 15 and p5_age <= 64
      group by wardid;
    """)


In [62]:
# Create base (general case) for 15-64 age range
cur.execute("""
    create table if not exists generalcase1564 as 
      select *
      from censuslabor
      where p2_membership <= 2
        and p5_age >= 15 and p5_age <= 64; 
    """)

In [63]:
# Define repeated code to run on specific case

def sharedSQL():
    
    cur.execute("drop table if exists censuswardpopthiscase;")
    cur.execute("""
        create temporary table censuswardpopthiscase as
          select wardid, count(wardid) as popthiscase
          from thiscase
          group by wardid;
    """)

    cur.execute("drop table if exists censuswardlf7days;")
    cur.execute("""
        create temporary table censuswardlf7days as
          select wardid, count(wardid) as lf7days
          from thiscase
          where p31_activity_last_7_days >= 1 and p31_activity_last_7_days <= 7
          group by wardid;
        """)

    cur.execute("drop table if exists censuswardlf12months;")
    cur.execute("""
        create temporary table censuswardlf12months as
          select wardid, count(wardid) as lf12months
          from thiscase
          where p32_activity_last_12_months >= 1 and p32_activity_last_12_months <= 7
          group by wardid;
        """)
    
    cur.execute("drop table if exists censuswardempl7days;")
    cur.execute("""   
        create temporary table censuswardempl7days as
          select wardid, count(wardid) as empl7days
          from thiscase
          where p31_activity_last_7_days >= 1 and p31_activity_last_7_days <= 6
          group by wardid;
        """)
    
    cur.execute("drop table if exists censuswardempl12months;")
    cur.execute("""   
        create temporary table censuswardempl12months as
          select wardid, count(wardid) as empl12months
          from thiscase
          where p32_activity_last_12_months >= 1 and p32_activity_last_12_months <= 6
          group by wardid;
        """)
    
    cur.execute("drop table if exists censuswardunem7days;")
    cur.execute("""   
        create temporary table censuswardunem7days as
          select wardid, count(wardid) as unem7days
          from thiscase
          where p31_activity_last_7_days = 7
          group by wardid;
        """)
    
    cur.execute("drop table if exists censuswardunem12months;")
    cur.execute("""   
        create temporary table censuswardunem12months as
          select wardid, count(wardid) as unem12months
          from thiscase
          where p32_activity_last_12_months = 7
          group by wardid;
        """)
    
    cur.execute("drop table if exists final;")
    cur.execute("""   
        create table final as
          select dist, const, ward, wardid, population1564, popthiscase,
            lf7days, lf12months, empl7days, empl12months, unem7days, unem12months
          from censuswardcounts1564
          full outer join censuswardpopthiscase using(wardid)
          full outer join censuswardlf7days using(wardid)
          full outer join censuswardlf12months using(wardid)
          full outer join censuswardempl7days using(wardid)
          full outer join censuswardempl12months using(wardid)
          full outer join censuswardunem7days using(wardid)
          full outer join censuswardunem12months using(wardid);
        """)

    cur.execute("drop table if exists censuswardunem7days;")
    cur.execute("drop table if exists censuswardempl7days;")
    cur.execute("drop table if exists censuswardunem12months;")
    cur.execute("drop table if exists censuswardempl12months;")
    cur.execute("drop table if exists censuswardlf12months;")
    cur.execute("drop table if exists censuswardlf7days;")
    cur.execute("drop table if exists censuswardpopthiscase;")
    cur.execute("drop table if exists thiscase;")

In [54]:
# Totals for population:

# Create specific case
cur.execute("drop table if exists thiscase;")
cur.execute("""
    create temporary table thiscase as 
      select *
      from generalcase1564; 
    """)

# Run shared code
sharedSQL()

# Create final data set for this case and clean up
cur.execute("drop table if exists censuswardLaborAll1564;")
cur.execute("""
    create table censuswardLaborAll1564 as 
      select *
      from final;
      drop table final;
    """)
conn.commit()

In [55]:
# Totals for the unskilled population
"""
There is a 1:1 match between p28_highest_level <=13 and p_29_high_vocation == 1
and p28 >= 14 and p_29 >= 2
Also, diploma and certificate are post secondary.
0 is none
<= 6 is none or primary incomplete
grade 7 is primary complete
8 - 12 is secondary, with 12 secondary complete
13 is some college
"""

# Create specific case
cur.execute("drop table if exists thiscase;")
cur.execute("""
    create temporary table thiscase as 
      select *
      from generalcase1564
      where p28_highest_level <= 6;
    """)

# Run shared code
sharedSQL()

# Create final data set for this case and clean up
cur.execute("drop table if exists censuswardLaborUnskilled1564;")
cur.execute("""
    create table censuswardLaborUnskilled1564 as 
      select *
      from final;
      drop table final;
    """)
conn.commit()

In [64]:
# Totals for the skilled population (see unskilled cell for explanations)

# Create specific case
cur.execute("drop table if exists thiscase;")
cur.execute("""
    create temporary table thiscase as 
      select *
      from generalcase1564
      where p29_high_vocation > 1;
    """)

# Run shared code
sharedSQL()

# Create final data set for this case and clean up
cur.execute("drop table if exists censuswardLaborSkilled1564;")
cur.execute("""
    create table censuswardLaborSkilled1564 as 
      select *
      from final;
      drop table final;
    """)
conn.commit()


In [66]:
# Totals for the semiskilled population (see unskilled cell for explanations)

# Create specific case
cur.execute("drop table if exists thiscase;")
cur.execute("""
    create temporary table thiscase as 
      select *
      from generalcase1564
      where (p28_highest_level >=7 and p28_highest_level <= 13);
    """)

# Run shared code
sharedSQL()

# Create final data set for this case and clean up
cur.execute("drop table if exists censuswardLaborSemiskilled1564;")
cur.execute("""
    create table censuswardLaborSemiskilled1564 as 
      select *
      from final;
      drop table final;
    """)
conn.commit()


In [68]:
# Totals for male population

# Create specific case
cur.execute("drop table if exists thiscase;")
cur.execute("""
    create temporary table thiscase as 
      select *
      from generalcase1564
      where p4_sex = 1;
    """)

# Run shared code
sharedSQL()

# Create final data set for this case and clean up
cur.execute("drop table if exists censuswardLaborMale1564;")
cur.execute("""
    create table censuswardLaborMale1564 as 
      select *
      from final;
      drop table final;
    """)
conn.commit()


In [69]:
# Totals for female population

# Create specific case
cur.execute("drop table if exists thiscase;")
cur.execute("""
    create temporary table thiscase as 
      select *
      from generalcase1564
      where p4_sex = 2;
    """)

# Run shared code
sharedSQL()

# Create final data set for this case and clean up
cur.execute("drop table if exists censuswardLaborFemale1564;")
cur.execute("""
    create table censuswardLaborFemale1564 as 
      select *
      from final;
      drop table final;
    """)
conn.commit()


In [71]:
# Totals for population aged 15-24

# Create specific case
cur.execute("drop table if exists thiscase;")
cur.execute("""
    create temporary table thiscase as 
      select *
      from generalcase1564
      where p5_age <= 24;
    """)

# Run shared code
sharedSQL()

# Create final data set for this case and clean up
cur.execute("drop table if exists censuswardLabor1524;")
cur.execute("""
    create table censuswardLabor1524 as 
      select *
      from final;
      drop table final;
    """)
conn.commit()


In [72]:
# Totals for population aged 25-64

# Create specific case
cur.execute("drop table if exists thiscase;")
cur.execute("""
    create temporary table thiscase as 
      select *
      from generalcase1564
      where p5_age >= 25;
    """)

# Run shared code
sharedSQL()

# Create final data set for this case and clean up
cur.execute("drop table if exists censuswardLabor2564;")
cur.execute("""
    create table censuswardLabor2564 as 
      select *
      from final;
      drop table final;
    """)
conn.commit()


In [73]:
# Totals for the agriculture 

"""
11 - 32 "Agriculture" 
51 - 99 "Mining" 
101 - 332 "Manufacturing" 
351 - 390 "Public utilities" 
410 - 439 "Construction"  
451 - 479 "Commerce" 
491 - 639 "Transport and Comnunications" 
641 - 829 "Financial and Business Services" 
841 - 843 "Public Administration" 
851 - 990 "Other Services, Unspecified"
"""

# Create specific case
cur.execute("drop table if exists thiscase;")
cur.execute("""
    create temporary table thiscase as 
      select *
      from generalcase1564
      where p35_industry >= 11 and p35_industry <= 32;
    """)

# Run shared code
sharedSQL()

# Create final data set for this case and clean up
cur.execute("drop table if exists censuswardLaborAgriculture1564;")
cur.execute("""
    create table censuswardLaborAgriculture1564 as 
      select *
      from final;
      drop table final;
    """)
conn.commit()


In [75]:
# Totals for the mining (see cell for agriculture for table)

# Create specific case
cur.execute("drop table if exists thiscase;")
cur.execute("""
    create temporary table thiscase as 
      select *
      from generalcase1564
      where p35_industry >= 51 and p35_industry <= 99;
    """)

# Run shared code
sharedSQL()

# Create final data set for this case and clean up
cur.execute("drop table if exists censuswardLaborMining1564;")
cur.execute("""
    create table censuswardLaborMining1564 as 
      select *
      from final;
      drop table final;
    """)
conn.commit()


In [77]:
# Totals for the manufacturing (see cell for agriculture for table)

# Create specific case
cur.execute("drop table if exists thiscase;")
cur.execute("""
    create temporary table thiscase as 
      select *
      from generalcase1564
      where p35_industry >= 101 and p35_industry <= 332;
    """)

# Run shared code
sharedSQL()

# Create final data set for this case and clean up
cur.execute("drop table if exists censuswardLaborManufacture1564;")
cur.execute("""
    create table censuswardLaborManufacture1564 as 
      select *
      from final;
      drop table final;
    """)
conn.commit()


In [79]:
# Totals for the other industries (see cell for agriculture for table)

# Create specific case
cur.execute("drop table if exists thiscase;")
cur.execute("""
    create temporary table thiscase as 
      select *
      from generalcase1564
      where p35_industry >= 351;
    """)

# Run shared code
sharedSQL()

# Create final data set for this case and clean up
cur.execute("drop table if exists censuswardLaborOtherIndustry1564;")
cur.execute("""
    create table censuswardLaborOtherIndustry1564 as 
      select *
      from final;
      drop table final;
    """)
conn.commit()


In [81]:
## Cleanup

cur.execute("drop table if exists censuswardcounts;")
cur.execute("drop table if exists censuswardLaborAll;")
cur.execute("drop table if exists censuswardLaborUnskilled;")
cur.execute("drop table if exists censuswardLaborSkilled;")
cur.execute("drop table if exists censuswardLaborSemiskilled;")
cur.execute("drop table if exists censuswardLaborMale;")
cur.execute("drop table if exists censuswardLaborFemale;")
cur.execute("drop table if exists censuswardLaborAgriculture;")
cur.execute("drop table if exists censuswardLaborMining;")
cur.execute("drop table if exists censuswardLaborManufacture;")
cur.execute("drop table if exists censuswardLaborOtherIndustry;")


In [82]:
conn.commit()

In [83]:
cur.close()
conn.close()